In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import autocontouring
sys.path[0] = str(Path(sys.path[0]).parent)
from experiments.utils.metrics import calculate_metrics


/export/scratch3/grewal/OAR_segmentation/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
usage: ipykernel_launcher.py [-h] [--env-file ENV_FILE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"19d59eb8-433e-4a33-a746-a806dde5a373" --shell=9007 --transport="tcp" --iopub=9009 --f=/export/scratch1/home/grewal/.local/share/jupyter/runtime/kernel-v2-3288121yTYBe03hQYKk.json


SystemExit: 2

/export/scratch3/grewal/OAR_segmentation/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
def process_data(patient_id, patient_path, output_path):
	class_names = ["background", "bowel_bag", "bladder", "hip", "rectum"]
	# read dicoms and obs1, obs2 delineations
	image = np.load(os.path.join(patient_path, "image.npz"))["volume"]
	seg_obs1 = np.load(os.path.join(patient_path, "observer1.npz"))["mask_volume"]
	seg_obs2 = np.load(os.path.join(patient_path, "observer2.npz"))["mask_volume"]

	# obtain segmentation mask from deep learning model
	seg_auto = autocontouring.main(image)

	# vizualization

	# compute metrics obs1 and auto, obs2 and auto, obs1 and obs2
	test_results = []
	metrics12 = calculate_metrics(seg_obs1, seg_obs2, class_names=class_names)
	print("metrics12: ", metrics12)
	dice = metrics12[3]
	for class_no, classname in enumerate(class_names):
		test_results.append({
			"patient_id": patient_id,
			"classname": classname,
			"observer_pair": "d12",
			"dice": dice[class_no]
		}
		)

	metrics1auto = calculate_metrics(seg_obs1, seg_auto, class_names=class_names)
	print("metrics1auto: ", metrics1auto)
	dice = metrics1auto[3]
	for class_no, classname in enumerate(class_names):
		test_results.append({
			"patient_id": patient_id,
			"classname": classname,
			"observer_pair": "d1auto",
			"dice": dice[class_no]
		}
		)

	metrics2auto = calculate_metrics(seg_obs2, seg_auto, class_names=class_names)
	print("metrics2auto: ", metrics2auto)
	dice = metrics2auto[3]
	for class_no, classname in enumerate(class_names):
		test_results.append({
			"patient_id": patient_id,
			"classname": classname,
			"observer_pair": "d2auto",
			"dice": dice[class_no]
		}
		)

	return test_results

In [ ]:
root_path = "../outputs/auto_vs_manual_data"
output_path = "../outputs/auto_vs_manual_results"

dicom_patient_ids = [1, 3, 6, 2, 4, 5]
# dicom_patient_ids = [3]
all_results = []
for patient_id in dicom_patient_ids:
	print("Patient no: ", patient_id)
	patient_path = os.path.join(root_path, "{0:03d}_MODIR_CERVIX_PATIENT/{0:03d}_CT_EBRT".format(patient_id))
	results = process_data(patient_id, patient_path, output_path)
	all_results.extend(results)
	print("")

df = pd.DataFrame.from_records(all_results)
print(df.head())